# Evaluations
<br>
Notebook for evaluating models.

In [1]:
import torch
from src.utils import *
import sys
import csv
import re
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, auc

In [7]:
probs_resnet_clean_csv = '../eval/prob_resnet_spec_eval.csv'
probs_LCNN_clean_csv = '../eval/prob_LCNN_spec_eval.csv'
probs_SENet_clean_csv = '../eval/prob_SENet_spec_eval.csv'

probs_resnet_clean_mag_csv = '../eval/prob_ResNet_spec_eval_mag.csv'
probs_SENet_clean_mag_csv = '../eval/prob_SENet_spec_eval_mag.csv'

probs_resnet_clean = pd.read_csv(probs_resnet_clean_csv, header=0, engine='python')
probs_LCNN_clean = pd.read_csv(probs_LCNN_clean_csv, header=0, engine='python')
probs_SENet_clean = pd.read_csv(probs_SENet_clean_csv, header=0, engine='python')

probs_resnet_clean_mag = pd.read_csv(probs_resnet_clean_mag_csv, header=0, engine='python')
probs_SENet_clean_mag = pd.read_csv(probs_SENet_clean_mag_csv, header=0, engine='python')

In [8]:
len(probs_resnet_clean_mag)

71237

In [9]:
if len(probs_LCNN_clean) == len(probs_SENet_clean) == len(probs_resnet_clean_mag) == len(probs_SENet_clean_mag):
    print(f'OK. Lengths are the same: {len(probs_resnet_clean_mag)-1}')
else:
    sys.exit('Not OK. Lengths are not the same')

OK. Lengths are the same: 71236


In [10]:
# ground truth labels of the evaluation dataset (ASVSpoof2019)
config_path_resnet = '../config/residualnet_train_config.yaml'
config_resnet = read_yaml(config_path_resnet)
df_eval = pd.read_csv(os.path.join('..', config_resnet['df_eval_path']))

***
## Extract predicted labels in the same order as in df_eval_19

In [11]:
def extract_id(file_path):
    match = re.search(r'LA_E_(\d+)', file_path)
    if match:
        return match.group(1)
    return None
    
    
def pred_probabilities(file2_path):
    # read df_eval_19
    file1_path = '../data/df_eval_19.csv'
    
    file1_ids = []
    with open(file1_path, 'r') as file1:
        csv_reader = csv.reader(file1)
        for row in csv_reader:
            file_id = extract_id(row[1])
            if file_id:
                file1_ids.append(file_id)
    
    # read second file and store data in a dictionary
    file2_data = {}
    with open(file2_path, 'r') as file2:
        csv_reader = csv.reader(file2)
        for row in csv_reader:
            file_id = extract_id(row[0])
            if file_id:
                file2_data[file_id] = (float(row[1]), float(row[2]))
                
    output_array = []
    for file_id in file1_ids:
        if file_id in file2_data:
            col2, col3 = file2_data[file_id]
            output_array.append(0 if col2>col3 else 1)
                
    return output_array

***
## Accuracies on clean dataset

In [12]:
pred_labels_clean_resnet = pred_probabilities(file2_path=probs_resnet_clean_csv)
pred_labels_clean_LCNN = pred_probabilities(file2_path=probs_LCNN_clean_csv)
pred_labels_clean_SeNet = pred_probabilities(file2_path=probs_SENet_clean_csv)

In [13]:
pred_labels_clean_mag_resnet = pred_probabilities(file2_path=probs_resnet_clean_mag_csv)
pred_labels_clean_mag_SeNet = pred_probabilities(file2_path=probs_SENet_clean_mag_csv)

In [14]:
GT_labels = df_eval.iloc[:,-1].tolist()
print(len(GT_labels))

71237


***
## Unbalanced accuracies for ALL MODELS on clean dataset

In [17]:
UA_resnet_clean = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_resnet)
UA_LCNN_clean = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_LCNN)
UA_SeNet_clean = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_SeNet)
UA_resnet_clean_mag = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_mag_resnet)
UA_SeNet_clean_mag = accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_mag_SeNet)

print(f'The unb.acc. for clean dataset for ResNet is {UA_resnet_clean*100:.2f}%\n'
      f'The unb.acc. for clean dataset for LCNN is {UA_LCNN_clean*100:.2f}%\n'
      f'The unb.acc. for clean dataset for SENet is {UA_SeNet_clean*100:.2f}%\n\n'
      f'The unb.acc. for clean dataset for ResNet (mag spec) is {UA_resnet_clean_mag*100:.2f}%\n'
      f'The unb.acc. for clean dataset for SENet (mag spec) is {UA_SeNet_clean_mag*100:.2f}%')

The unb.acc. for clean dataset for ResNet is 77.14%
The unb.acc. for clean dataset for LCNN is 52.85%
The unb.acc. for clean dataset for SENet is 82.80%

The unb.acc. for clean dataset for ResNet (mag spec) is 88.26%
The unb.acc. for clean dataset for SENet (mag spec) is 80.11%


***
## Balanced accuracy for ALL MODELS on clean dataset

In [19]:
BA_resnet_clean = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_resnet)
BA_LCNN_clean = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_LCNN)
BA_SeNet_clean = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_SeNet)
BA_resnet_clean_mag = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_mag_resnet)
BA_SeNet_clean_mag = balanced_accuracy_score(y_true=GT_labels, y_pred=pred_labels_clean_mag_SeNet)

print(f'The balanced acc. for clean dataset for ResNet is {BA_resnet_clean*100:.2f}%\n'
      f'The balanced acc. for clean dataset for LCNN is {BA_LCNN_clean*100:.2f}%\n'
      f'The balanced acc. for clean dataset for SENet is {BA_SeNet_clean*100:.2f}%\n\n'
      f'The balanced acc. for clean dataset for ResNet (mag spec) is {BA_resnet_clean_mag*100:.2f}%\n'
      f'The balanced acc. for clean dataset for SENet (mag spec) is {BA_SeNet_clean_mag*100:.2f}%')

The balanced acc. for clean dataset for ResNet is 84.21%
The balanced acc. for clean dataset for LCNN is 72.98%
The balanced acc. for clean dataset for SENet is 88.35%

The balanced acc. for clean dataset for ResNet (mag spec) is 90.52%
The balanced acc. for clean dataset for SENet (mag spec) is 87.38%


***
## Balanced accuracies of LCNN and SENet on FGSM attack performed with ResNet 



In [1]:
def compute_and_print_bal_accuracy(values, model, attack, at_model, GT_labels, flag=None):
    
    for i in values:
        if flag == None:
            epsilon_str = str(i).replace('.', 'dot')
            csv_file = f'../eval/prob_{model}_{attack}_{at_model}_{epsilon_str}.csv'
            probs = pred_probabilities(file2_path=csv_file)
            BA = balanced_accuracy_score(y_true=GT_labels, y_pred=probs)
            print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {BA*100:.2f}%')
        elif flag == 'ResLow':
            epsilon_str = str(i).replace('.', 'dot')
            csv_file = f'../eval/prob_resnet_spec_eval_FGSM_{epsilon_str}.csv'
            probs = pred_probabilities(file2_path=csv_file)
            BA = balanced_accuracy_score(y_true=GT_labels, y_pred=probs)
            print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {BA*100:.2f}%')
        elif flag == 'Res1':
            epsilon_str = str(i).replace('.', 'dot')
            csv_file = f'../eval/prob_{model}_{attack}_{at_model}_{epsilon_str}_1.csv'
            probs = pred_probabilities(file2_path=csv_file)
            BA = balanced_accuracy_score(y_true=GT_labels, y_pred=probs)
            print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {BA*100:.2f}%')
        
            
    # if model == 'ResNet' and at_model == 'ResNet' and attack == 'FGSM':
    #     for i in values:
    #         epsilon_str = str(i).replace('.', 'dot')
    #         csv_file = f'../eval/prob_resnet_spec_eval_FGSM_{epsilon_str}.csv'
    #         probs = pred_probabilities(file2_path=csv_file)
    #         BA = balanced_accuracy_score(y_true=GT_labels, y_pred=probs)
    #     
    #         print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {BA*100:.2f}%')
    # else:
    #     for i in values:
    #         epsilon_str = str(i).replace('.', 'dot')
    #         csv_file = f'../eval/prob_{model}_{attack}_{at_model}_{epsilon_str}.csv'
    #         probs = pred_probabilities(file2_path=csv_file)
    #         probs = pred_probabilities(file2_path=csv_file)
    #         BA = balanced_accuracy_score(y_true=GT_labels, y_pred=probs)
    #         print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {BA*100:.2f}%')
    
def compute_and_print_unbal_accuracy(values, model, attack, at_model, GT_labels):
    for i in values:
        epsilon_str = str(i).replace('.', 'dot')
        csv_file = f'../eval/prob_{model}_{attack}_{at_model}_{epsilon_str}.csv'
        probs = pred_probabilities(file2_path=csv_file)
        probs = pred_probabilities(file2_path=csv_file)
        A = accuracy_score(y_true=GT_labels, y_pred=probs)
        print(f'Unbalanced accuracy for {model} on {attack} {at_model} for eps={i} is {A*100:.2f}%')
    
    
    # if model != 'ResNet':
    #     for i in values:
    #         epsilon_str = str(i).replace('.', 'dot')
    #         csv_file = f'../eval/prob_{model}_{attack}_{at_model}_{epsilon_str}.csv'
    #         probs = pred_probabilities(file2_path=csv_file)
    #         UBA = accuracy_score(y_true=GT_labels, y_pred=probs)
    #     
    #         print(f'Unbalanced accuracy for {model} on {attack} {at_model} for eps={i} is {UBA*100:.2f}%')
    # elif model == 'ResNet' and at_model == 'ResNet':
    #     for i in values:
    #         epsilon_str = str(i).replace('.', 'dot')
    #         csv_file = f'../eval/prob_resnet_spec_eval_{attack}_{epsilon_str}.csv'
    #         probs = pred_probabilities(file2_path=csv_file)
    #         UBA = accuracy_score(y_true=GT_labels, y_pred=probs)
    #     
    #         print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {UBA*100:.2f}%')
    # elif model == 'ResNet' and at_model == 'SENet':
    #     for i in values:
    #         epsilon_str = str(i).replace('.', 'dot')
    #         csv_file = f'../eval/prob_ResNet_{attack}_{at_model}_{epsilon_str}.csv'
    #         probs = pred_probabilities(file2_path=csv_file)
    #         probs = pred_probabilities(file2_path=csv_file)
    #         UBA = accuracy_score(y_true=GT_labels, y_pred=probs)
    #         print(f'Balanced accuracy for {model} on {attack} {at_model} for eps={i} is {UBA*100:.2f}%')
    # 
    # else:
    #     print('TODO')    
    

In [12]:
compute_and_print_bal_accuracy([0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 3.0], 'ResNet', 'FGSM', 'ResNet', GT_labels, flag='ResLow')

Balanced accuracy for ResNet on FGSM ResNet for eps=0.2 is 77.53%
Balanced accuracy for ResNet on FGSM ResNet for eps=0.4 is 70.05%
Balanced accuracy for ResNet on FGSM ResNet for eps=0.6 is 62.14%
Balanced accuracy for ResNet on FGSM ResNet for eps=0.8 is 53.36%
Balanced accuracy for ResNet on FGSM ResNet for eps=1.0 is 43.81%
Balanced accuracy for ResNet on FGSM ResNet for eps=2.0 is 13.89%
Balanced accuracy for ResNet on FGSM ResNet for eps=3.0 is 6.63%


In [13]:
compute_and_print_bal_accuracy([0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 3.0], 'LCNN', 'FGSM', 'ResNet', GT_labels)

Balanced accuracy for LCNN on FGSM ResNet for eps=0.2 is 69.04%
Balanced accuracy for LCNN on FGSM ResNet for eps=0.4 is 68.48%
Balanced accuracy for LCNN on FGSM ResNet for eps=0.6 is 67.94%
Balanced accuracy for LCNN on FGSM ResNet for eps=0.8 is 67.41%
Balanced accuracy for LCNN on FGSM ResNet for eps=1.0 is 66.91%
Balanced accuracy for LCNN on FGSM ResNet for eps=2.0 is 64.53%
Balanced accuracy for LCNN on FGSM ResNet for eps=3.0 is 62.24%


In [14]:
compute_and_print_bal_accuracy([0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 3.0], 'SENet', 'FGSM', 'ResNet', GT_labels)

Balanced accuracy for SENet on FGSM ResNet for eps=0.2 is 88.00%
Balanced accuracy for SENet on FGSM ResNet for eps=0.4 is 87.56%
Balanced accuracy for SENet on FGSM ResNet for eps=0.6 is 87.10%
Balanced accuracy for SENet on FGSM ResNet for eps=0.8 is 86.54%
Balanced accuracy for SENet on FGSM ResNet for eps=1.0 is 85.89%
Balanced accuracy for SENet on FGSM ResNet for eps=2.0 is 83.29%
Balanced accuracy for SENet on FGSM ResNet for eps=3.0 is 80.79%


***
## SENet FGSM attack


In [9]:
compute_and_print_bal_accuracy([0.2, 1.0, 2.0, 3.0], 'SENet', 'FGSM', 'SENet', GT_labels)

Balanced accuracy for SENet on FGSM SENet for eps=0.2 is 87.04%
Balanced accuracy for SENet on FGSM SENet for eps=1.0 is 80.19%
Balanced accuracy for SENet on FGSM SENet for eps=2.0 is 71.97%
Balanced accuracy for SENet on FGSM SENet for eps=3.0 is 65.29%


***
## SENet FGSM UNCUT attack

In [12]:
compute_and_print_bal_accuracy([1.0, 2.0, 3.0], 'SENet', 'FGSM_UNCUT', 'SENet', GT_labels)
compute_and_print_unbal_accuracy([1.0, 2.0, 3.0], 'SENet', 'FGSM_UNCUT', 'SENet', GT_labels)

Balanced accuracy for SENet on FGSM_UNCUT SENet for eps=1.0 is 79.24%
Balanced accuracy for SENet on FGSM_UNCUT SENet for eps=2.0 is 70.04%
Balanced accuracy for SENet on FGSM_UNCUT SENet for eps=3.0 is 62.76%
Unbalanced accuracy for SENet on FGSM_UNCUT SENet for eps=1.0 is 70.26%
Unbalanced accuracy for SENet on FGSM_UNCUT SENet for eps=2.0 is 59.52%
Unbalanced accuracy for SENet on FGSM_UNCUT SENet for eps=3.0 is 52.61%


***
## Transferability of SENet FGSM on ResNet

In [16]:
compute_and_print_bal_accuracy([1.0, 2.0, 3.0], 'ResNet', 'FGSM', 'SENet', GT_labels)

Balanced accuracy for ResNet on FGSM SENet for eps=1.0 is 83.91%
Balanced accuracy for ResNet on FGSM SENet for eps=2.0 is 83.32%
Balanced accuracy for ResNet on FGSM SENet for eps=3.0 is 82.65%


## Transferability of SENet FGSM on LCNN

In [25]:
compute_and_print_bal_accuracy([1.0], 'LCNN', 'FGSM', 'SENet', GT_labels)

Balanced accuracy for LCNN on FGSM SENet for eps=1.0 is 69.07%


***
## Transferability of LCNN FGSM on ResNet

In [22]:
compute_and_print_bal_accuracy([2.0, 3.0], 'ResNet', 'FGSM', 'LCNN', GT_labels)

Balanced accuracy for ResNet on FGSM LCNN for eps=2.0 is 82.96%
Balanced accuracy for ResNet on FGSM LCNN for eps=3.0 is 82.18%


## Transferability of LCNN FGSM on SENet

In [24]:
compute_and_print_bal_accuracy([2.0], 'SENet', 'FGSM', 'LCNN', GT_labels)

Balanced accuracy for SENet on FGSM LCNN for eps=2.0 is 86.34%


***
## FGSM on LCNN

In [26]:
compute_and_print_bal_accuracy([2.0, 3.0], 'LCNN', 'FGSM', 'LCNN', GT_labels)

Balanced accuracy for LCNN on FGSM LCNN for eps=2.0 is 66.75%
Balanced accuracy for LCNN on FGSM LCNN for eps=3.0 is 65.65%


***
# FGSM (double test) on ResNet
Basically I re-run the FGSM attack using the same code I used for LCNN and SENet because with those two models I was getting low success rates.

In [15]:
compute_and_print_unbal_accuracy([1.0,], 'ResNet', 'FGSM', 'ResNet', GT_labels)
# with 1.0 but you have to add a _1 to the csv name

Balanced accuracy for ResNet on FGSM ResNet for eps=1.0 is 41.51%


In [19]:
compute_and_print_unbal_accuracy([2.0, 3.0], 'ResNet', 'FGSM', 'ResNet', GT_labels)

Balanced accuracy for ResNet on FGSM ResNet for eps=2.0 is 22.92%
Balanced accuracy for ResNet on FGSM ResNet for eps=3.0 is 11.86%


***
## FGSM uncut on LCNN

In [21]:
compute_and_print_bal_accuracy([3.0], 'LCNN', 'FGSM_UNCUT', 'LCNN', GT_labels)

Balanced accuracy for LCNN on FGSM_UNCUT LCNN for eps=3.0 is 65.81%


***
## BIM classic (no grad cut, yes audio cut) on SENet

In [27]:
compute_and_print_bal_accuracy([1.0], 'SENet', 'BIM', 'SENet', GT_labels)

Balanced accuracy for SENet on BIM SENet for eps=1.0 is 85.27%
